In [10]:

# Import libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout
from keras.metrics import Precision, Recall, AUC
from keras.callbacks import EarlyStopping

from google.colab import drive
drive.mount('/content/drive')

df= pd.read_csv('/content/drive/My Drive/Colab Notebooks/Phishing_Legitimate_full.csv')

# Check structure
print(df.head())
print(df.columns)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
   id  NumDots  SubdomainLevel  PathLevel  UrlLength  NumDash  \
0   1        3               1          5         72        0   
1   2        3               1          3        144        0   
2   3        3               1          2         58        0   
3   4        3               1          6         79        1   
4   5        3               0          4         46        0   

   NumDashInHostname  AtSymbol  TildeSymbol  NumUnderscore  ...  \
0                  0         0            0              0  ...   
1                  0         0            0              2  ...   
2                  0         0            0              0  ...   
3                  0         0            0              0  ...   
4                  0         0            0              0  ...   

   IframeOrFrame  MissingTitle  ImagesOnlyInForm  SubdomainLevelRT  \
0      

In [12]:
# Replace 'ClassLabel' with your actual label column name if different
X = df.drop('CLASS_LABEL', axis=1)
y = df['CLASS_LABEL']

# Normalize feature values
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.3, random_state=42)

In [13]:
# Reshape to 3D
X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)
X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], 1)

In [14]:
model = Sequential()
model.add(LSTM(128, input_shape=(X_train.shape[1], 1)))
model.add(Dropout(0.3))  # Helps prevent overfitting
model.add(Dense(1, activation='sigmoid'))  # Binary classification

/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [15]:
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy', Precision(), Recall(), AUC()])

In [23]:
import time

start = time.time()

history = model.fit(X_train, y_train,
                    validation_split=0.2,
                    epochs=50,
                    batch_size=64,
                    callbacks=[earlystop],
                    verbose=2)

end = time.time()

print(f"Training time: {end - start:.2f} seconds")

Epoch 1/50
88/88 - 5s - 56ms/step - accuracy: 0.9818 - auc: 0.9984 - loss: 0.0479 - precision: 0.9894 - recall: 0.9737 - val_accuracy: 0.9964 - val_auc: 0.9994 - val_loss: 0.0276 - val_precision: 0.9971 - val_recall: 0.9957
Epoch 2/50
88/88 - 4s - 47ms/step - accuracy: 0.9925 - auc: 0.9992 - loss: 0.0255 - precision: 0.9946 - recall: 0.9903 - val_accuracy: 0.9993 - val_auc: 0.9999 - val_loss: 0.0105 - val_precision: 1.0000 - val_recall: 0.9986
Epoch 3/50
88/88 - 5s - 60ms/step - accuracy: 0.9962 - auc: 0.9999 - loss: 0.0132 - precision: 0.9978 - recall: 0.9946 - val_accuracy: 0.9879 - val_auc: 0.9999 - val_loss: 0.0279 - val_precision: 0.9773 - val_recall: 0.9986
Epoch 4/50
88/88 - 9s - 103ms/step - accuracy: 0.9779 - auc: 0.9956 - loss: 0.0706 - precision: 0.9864 - recall: 0.9686 - val_accuracy: 0.9971 - val_auc: 0.9998 - val_loss: 0.0213 - val_precision: 0.9985 - val_recall: 0.9957
Epoch 5/50
88/88 - 5s - 57ms/step - accuracy: 0.9961 - auc: 0.9997 - loss: 0.0174 - precision: 0.9978 -

In [21]:
# Predict on test set
y_pred_prob = model.predict(X_test)
y_pred = (y_pred_prob > 0.5).astype(int)

# Evaluation
print(classification_report(y_test, y_pred))
print("AUC Score:", roc_auc_score(y_test, y_pred_prob))

# Confusion Matrix
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))


94/94 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      1463
           1       1.00      1.00      1.00      1537

    accuracy                           1.00      3000
   macro avg       1.00      1.00      1.00      3000
weighted avg       1.00      1.00      1.00      3000

AUC Score: 0.9996851417595862
Confusion Matrix:
[[1463    0]
 [   2 1535]]


In [22]:
print("Total trainable parameters:", model.count_params())

Total trainable parameters: 66689


In [27]:
!git clone https://github.com/JamesTan44181/DLI-Group-Assignment.git
%cd DLI-Group-Assignment

Cloning into 'DLI-Group-Assignment'...
remote: Enumerating objects: 13, done.
remote: Counting objects: 100% (13/13), done.
remote: Compressing objects: 100% (11/11), done.
remote: Total 13 (delta 3), reused 2 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (13/13), 953.87 KiB | 2.17 MiB/s, done.
Resolving deltas: 100% (3/3), done.
/content/DLI-Group-Assignment


In [30]:
!ls /content


DLI-Group-Assignment  drive  sample_data


In [29]:
import shutil
shutil.move('/content/Shao Ern Individual.ipynb', '/content/DLI-Group-Assignment/Shao Ern Individual.ipynb')


FileNotFoundError: [Errno 2] No such file or directory: '/content/Shao Ern Individual.ipynb'